In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from os import listdir
from os.path import isfile, join
from sklearn import datasets 
from sklearn.metrics import confusion_matrix 
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score

In [2]:
path="C:\\Users\\nilu1\\Desktop\\Code\\AndroidPattern\\Data\\Processed\\"
pathex = [f for f in listdir(path) if isfile(join(path, f))]
# for i in range(len(pathex)):
#     print(pathex[i])

In [3]:
for file in pathex:
    ni=file.rfind("_")+1
    n=int(file[ni:].split(".")[0])
    df=pd.read_csv(path+file)
    X = df.iloc[:, 0:n].values
    y = df.iloc[:, n].values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=2)
    gnb = GaussianNB().fit(X_train, y_train) 
    y_pred = gnb.predict(X_test)
    df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    print(file+" : "+str(f1_score(y_test, y_pred, average="micro")))
#     df

Acc_w_25_4vuni_all_9.csv : 0.11597374179431072
Acc_w_25_4v_all_9.csv : 0.1303395399780942
Acc_w_25_windowdelinit_all_75.csv : 0.1180327868852459
Acc_w_25_window_all_75.csv : 0.12131147540983607
Acc_w_30_4vuni_all_9.csv : 0.11706783369803063
Acc_w_30_4v_all_9.csv : 0.12595837897042717
Acc_w_30_windowdelinit_all_90.csv : 0.11584699453551912
Acc_w_30_window_all_90.csv : 0.12021857923497267


In [4]:
# sc = StandardScaler()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)
# sc = Normalizer()
# X_train = sc.fit_transform(X_train)
# X_test = sc.transform(X_test)

In [5]:
# non-uniform = 0.3636, 0.55(2 days)
# uniform = 0.3636
